In [ ]:
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import collections
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from collections import OrderedDict
import re
import gzip
from Bio import SeqIO

import torch.nn.functional as F
import torch.utils.data
import torch.nn as nn
from torch import relu, sigmoid
import torch.nn.modules.activation as activation
import matplotlib
import os
import sys
import copy
matplotlib.use('Agg')
from scipy.spatial import distance
from scipy.spatial.distance import cdist
from sklearn import metrics
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pickle
from Bio.SeqUtils import GC

import h5py
import kipoi
#import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Building the matrix

In [ ]:
#loading samples/regions/tf information
with gzip.open('../data/idx_files/regions_idx.pickle.gz', 'rb') as f:
    regions = pickle.load(f) #1817918
    
with gzip.open('../data/idx_files/samples_idx.pickle.gz', 'rb') as f:
    samples = pickle.load(f) #52
    
with gzip.open('../data/idx_files/tfs_idx.pickle.gz', 'rb') as f:
    tfs = pickle.load(f) #163
    
tfs = pd.Series(tfs).sort_values()
regions = pd.Series(regions).sort_values()

In [ ]:
#loading the matrices
data = np.load("../data/matrices/matrix2d.ReMap+UniBind.partial.npz")

for i in data.files:
    matrix2d_partial = data[i] #(1817918, 163)
    
data = np.load("../data/matrices/matrix2d.ReMap+UniBind.full.npz")

for i in data.files:
    matrix2d_full = data[i] #(1817918, 163)
    
data = np.load("../data/matrices/matrix2d.ReMap+UniBind.sparse.npz")

for i in data.files:
    matrix2d_sparse = data[i] #(1817918, 163)

In [ ]:
df_sparse = pd.DataFrame(data=matrix2d_sparse, index=regions.index, columns=tfs.index)

In [ ]:
df_sparse.shape

In [ ]:
#sort by ones
ones = {}
resolved = {}
for tf in list(df_sparse):
    ones[tf] = df_sparse[tf].dropna().sum()
    resolved[tf] = df_sparse[tf].dropna().shape[0]
    
ones = pd.Series(ones)
resolved = pd.Series(resolved)

In [ ]:
ones = ones.sort_values(ascending=False)
resolved = resolved.sort_values(ascending=False)

In [ ]:
fig = go.Figure([go.Bar(x=ones.index, y=ones.values)])

fig.update_layout(title='',
                   xaxis_title='TFs',
                   yaxis_title='Number of peaks',
                 plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_xaxes(tickfont=dict(size=5))
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.update_layout(title='',
                 font=dict(
                     family="Courier New, monospace",
                     size=18,
                     color="black"
                 ))

fig.update_layout(autosize=False,width=1600,height=800)

fig.show()

## Analysis of partial data

In [ ]:
df = pd.DataFrame(data=matrix2d_partial, index=regions.index, columns=tfs.index)
#df_sparse = pd.DataFrame(data=matrix2d_sparse, index=regions.index, columns=tfs.index)

with open("../data/Analyzed_TFs.txt", "w") as f:
    for tf in list(df):
        f.write(tf + "\n")
        
#copy of the original data frame
df_to_save = df.copy()

df_to_save.index = regions[df_to_save.index].values
df_to_save.index = df_to_save.index.map(str)

#saving the data frame for the later usage
with open('../data/final_df.pkl', 'wb') as f:
    pickle.dump(df_to_save, f)
    
#Count non-NA cells for each column or row.
counts = df.count()

#numbers of not nones (resolved)
fig = go.Figure([go.Bar(x=counts.index, y=counts.values)])

fig.update_layout(title='Majority of TFs dont have a lot of resolved regions',
                   xaxis_title='TFs',
                   yaxis_title='Number of peaks',
                 plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show()

In [ ]:
nan_sums = df.isna().sum()

#calculate the number of None regions per TF
nan_sums = df.isna().sum()

nan_perc = nan_sums/df.shape[0]*100
nan_perc = nan_perc.sort_values(ascending=True)

fig = go.Figure([go.Bar(x=list(nan_perc.index), y=nan_perc)])

fig['layout'].update(shapes=[{'type': 'line','y0':50,'y1': 50,'x0':nan_perc.index[0], 
                              'x1':nan_perc.index[-1],'xref':'x1','yref':'y1',
                              'line': {'color': 'red','width': 2.5}}])

fig.update_layout(title='Majority of TFs have more than 50% of regions as Nans',
                   xaxis_title='TFs',
                   yaxis_title='Percentage of Nones',
                 plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show()

## Analysis of sequences GC content

In [ ]:
fasta_sequences = SeqIO.parse(open("../data/sequences/sequences.200bp.fa"),'fasta')
    
all_gc_content = {}
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    
    #new format sequence file
    name = name.split(":")[0]
          
    all_gc_content[name] = GC(sequence.upper())
    
#get GC content for the specified TF
def get_gc_content(tf):
    tf_peaks = df[tf]
    tf_peaks = tf_peaks.dropna()
    
    res_gc_content = all_gc_content[regions[tf_peaks.index].values]
    
    return res_gc_content

all_gc_content = pd.Series(all_gc_content)

top5_gc_content = get_gc_content(nan_perc.index[:5].tolist()) 
top10_gc_content = get_gc_content(nan_perc.index[:10].tolist()) 
top20_gc_content = get_gc_content(nan_perc.index[:20].tolist())

top30_gc_content = get_gc_content(nan_perc.index[:30].tolist()) 
top40_gc_content = get_gc_content(nan_perc.index[:40].tolist()) 
top50_gc_content = get_gc_content(nan_perc.index[:50].tolist()) 

top60_gc_content = get_gc_content(nan_perc.index[:60].tolist()) 
top70_gc_content = get_gc_content(nan_perc.index[:70].tolist()) 
top80_gc_content = get_gc_content(nan_perc.index[:80].tolist()) 

top90_gc_content = get_gc_content(nan_perc.index[:90].tolist()) 
top100_gc_content = get_gc_content(nan_perc.index[:100].tolist()) 
top110_gc_content = get_gc_content(nan_perc.index[:110].tolist()) 
 
top120_gc_content = get_gc_content(nan_perc.index[:120].tolist()) 
top130_gc_content = get_gc_content(nan_perc.index[:130].tolist()) 
top140_gc_content = get_gc_content(nan_perc.index[:140].tolist()) 

top150_gc_content = get_gc_content(nan_perc.index[:150].tolist()) 
top160_gc_content = get_gc_content(nan_perc.index[:160].tolist()) 
alltf_gc_content = get_gc_content(nan_perc.index.tolist()) 

In [ ]:
#how many regions we have if we keep only most resolved TFs?
tf_num = [0,5,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170]
num_peaks = [len(all_gc_content), len(top5_gc_content), len(top10_gc_content),
            len(top20_gc_content), len(top30_gc_content), len(top40_gc_content),
            len(top50_gc_content), len(top60_gc_content), len(top70_gc_content),
            len(top80_gc_content), len(top90_gc_content), len(top100_gc_content),
            len(top110_gc_content), len(top120_gc_content), len(top130_gc_content),
            len(top140_gc_content), len(top150_gc_content), len(top160_gc_content), len(alltf_gc_content)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=tf_num, y=num_peaks,
                          line=dict(color='royalblue', width=4),
                          mode='lines', name = "New matrix"))

fig.update_layout(title='Numbers of peaks per TF',
                   xaxis_title='Number of TFs',
                   yaxis_title='Number of peaks',
                 plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show()

In [ ]:
#Density plot of GC content
fig = ff.create_distplot([all_gc_content,top5_gc_content,top10_gc_content,
                         top20_gc_content, top50_gc_content, top60_gc_content], 
                         ["All_peaks", "Top_5","Top_10", "Top_20", "Top_50", "Top_60"], 
                         colors=["blue", "green", "black", "pink", "darkblue", "crimson"],
                         show_rug=False, show_hist=False)

# Add title
fig.update_layout(title_text='GC distribution, new matrix', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.show()

In [ ]:
#select first 50 TFs from a sorted by number of resolved regions list
tf_peaks_50 = df[nan_perc.index[:50].tolist()]
tf_peaks_50 = tf_peaks_50.dropna() #(122428, 50)
tf_peaks_50.shape

In [ ]:
perc_50 = tf_peaks_50.sum(axis=0)/tf_peaks_50.shape[0]*100

fig = go.Figure([go.Bar(x=list(perc_50.index), y=perc_50)])

fig.update_layout(title_text='(50 TFs) The most represented TF (CTCF) has around 30% of positives',
                  xaxis_title='TFs',
                  yaxis_title='Percentage of 1s',
                  plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show()

In [ ]:
tf_sum_50 = tf_peaks_50.sum(axis=1)
tf_zerosum_50 = tf_sum_50[np.where(tf_sum_50 == 0)[0]]

print("Number of zero rows in the 50 df")
print(len(tf_zerosum_50))

In [ ]:
seq_names_50 = regions[tf_peaks_50.index].values
fasta_ids_50 = []
fasta_sequences_50 = {}

#ALL SEQUENCES
fasta_sequences = SeqIO.parse(open("../data/sequences/sequences.200bp.fa"),'fasta')
    
#get fasta sequences and delete NaNs    
with open("../data/sequences/tf_peaks_50_noN_unibind_remap.fa", "w") as f:
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        
        #new format sequence file
        name = name.split(":")[0]
        
        if int(name) in seq_names_50:
            #remove sequences with Ns
            if "N" in sequence.upper():
                print(name)
                continue
            else:
                fasta_ids_50.append(int(name))
                fasta_sequences_50[int(name)] = sequence.upper()
                f.write(">" + name + "\n")
                f.write(sequence.upper() + "\n")

In [ ]:
fasta_sequences_50 = pd.Series(fasta_sequences_50)

np.all(regions.values == np.array(range(1817918)))

In [ ]:
fasta_ids_50_noNs = regions[fasta_ids_50]

tf_peaks_50_noNs = tf_peaks_50.loc[fasta_ids_50_noNs.index,:] #remove Ns sequences
tf_peaks_50_noNs = tf_peaks_50_noNs.astype(int)
tf_peaks_50_noNs.index = fasta_ids_50_noNs[tf_peaks_50_noNs.index].values

tf_peaks_50_noNs.shape

In [ ]:
#save the matrix
with open('../data/tf_peaks_50_noNs_partial.pkl', 'wb') as f:
    pickle.dump(tf_peaks_50_noNs, f)
    
#save the fasta sequences
with open('../data/fasta_sequences_50_partial.pkl', 'wb') as f:
    pickle.dump(fasta_sequences_50, f)